<a href="https://colab.research.google.com/github/ellamcho/NB240/blob/main/MNIST/20250204_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tutorial from https://www.youtube.com/watch?v=OMDn66kM9Qc

In [9]:
# Load packages
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

In [10]:
# Define the model
model = nn.Sequential(
    nn.Linear(28 * 28, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
).cuda()

In [11]:
# Define the optimizer
params = model.parameters
optimizer = optim.SGD(params(),lr = 1e-2)

In [12]:
# Define loss
loss = nn.CrossEntropyLoss()

In [13]:
# Training sets

train_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
train, val = random_split(train_data, [55000, 5000]) # splits data randomly for training and then validation ("val")
train_loader = DataLoader(train, batch_size =32)
val_loader = DataLoader(val, batch_size = 32)

In [14]:
# Training and validation loops
num_epochs = 5
for epoch in range(num_epochs):
  losses =  list()
  accuracies = list()
  for batch in train_loader:
    x, y = batch

    # x: b x 1 x 28 x 28
    b = x.size(0)
    x = x.view(b, -1).cuda()

    # 1: Forward (l : logits)
    l = model(x)

    # 2: Compute the objective function
    J = loss(l, y.cuda())

    # 3: Clean the gradients
    model.zero_grad()
    #params.grad._zero()

    # 4: Accumulate the partial derivatives of J
    J.backward()
    # params.grad._sum(dJ/dparams)

    # 5: Step in the opposite direction of the gradient
    optimizer.step()

    losses.append(J.item())
    accuracies.append(y.eq(l.detach().argmax(dim=1).cpu()).float().mean())

  print(f'Epoch {epoch + 1}, train loss: {torch.tensor(losses).mean():2f}')
  print(f'training accuracy: {torch.tensor(accuracies).mean():.2f}')

    # with torch_nograd: params = eta - params.grad()

  losses =  list()
  for batch in val_loader:
    x, y = batch

    # x: b x 1 x 28 x 28
    b = x.size(0)
    x = x.view(b, -1).cuda()

    # 1: Forward (l : logits)
    with torch.no_grad():
      l = model(x)

    # 2: Compute the objective function
    J = loss(l, y.cuda())

    losses.append(J.item())
    accuracies.append(y.eq(l.detach().argmax(dim=1).cpu()).float().mean())

  print(f'Epoch {epoch + 1}, validation loss: {torch.tensor(losses).mean():2f}')
  print(f'training accuracy: {torch.tensor(accuracies).mean():.2f}')



Epoch 1, train loss: 1.251677
training accuracy: 0.67
Epoch 1, validation loss: 0.479242
training accuracy: 0.69
Epoch 2, train loss: 0.400200
training accuracy: 0.89
Epoch 2, validation loss: 0.334087
training accuracy: 0.89
Epoch 3, train loss: 0.320659
training accuracy: 0.91
Epoch 3, validation loss: 0.289010
training accuracy: 0.91
Epoch 4, train loss: 0.281065
training accuracy: 0.92
Epoch 4, validation loss: 0.258577
training accuracy: 0.92
Epoch 5, train loss: 0.250804
training accuracy: 0.93
Epoch 5, validation loss: 0.234990
training accuracy: 0.93
